Lab

In [1]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots

In [5]:
import statsmodels.api as sm

In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm

In [7]:
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)

In [8]:
dir()

['In',
 'MS',
 'Out',
 'VIF',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_exit_code',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'anova_lm',
 'exit',
 'get_ipython',
 'load_data',
 'np',
 'pd',
 'poly',
 'quit',
 'sm',
 'subplots',
 'summarize']

In [10]:
A = np.array([[2, 3], [5, 6]])
dir(A)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',

In [12]:
A.view()

array([[2, 3],
       [5, 6]])

In [13]:
boston = load_data('Boston')
boston.columns

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'lstat', 'medv'],
      dtype='object')

In [15]:
X = pd.DataFrame({'intercept': np.ones(boston.shape[0]), 'lstat': boston['lstat']})
X.head(5)

,intercept,lstat
0,1.0,4.98
1,1.0,9.14
2,1.0,4.03
3,1.0,2.94
4,1.0,5.33


In [16]:
y = boston['medv']
model = sm.OLS(y, X)

In [18]:
results = model.fit()
summarize(results)

,coef,std err,t,P>|t|
intercept,34.5538,0.563,61.415,0.0
lstat,-0.9500,0.039,-24.528,0.0


In [21]:
design = MS(['lstat'])
X = design.fit_transform(boston)
X.head(5)

,intercept,lstat
0,1.0,4.98
1,1.0,9.14
2,1.0,4.03
3,1.0,2.94
4,1.0,5.33


In [22]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     601.6
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           5.08e-88
Time:                        09:32:23   Log-Likelihood:                -1641.5
No. Observations:                 506   AIC:                             3287.
Df Residuals:                     504   BIC:                             3295.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     34.5538      0.563     61.415      0.000      33.448      35.659
lstat         -0.9500      0.039    -24.528      0.000      -1.026      -0.874
==============================================================================
Omnibus:                      137.043   Durbin-Watson:                   0.892
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              291.373
Skew:                           1.453   Prob(JB):                     5.36e-64
Kurtosis:                       5.319   Cond. No.                         29.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
new_predictions = results.get_prediction(X)
new_predictions.predicted_mean[:5]

array([29.8225951 , 25.87038979, 30.72514198, 31.76069578, 29.49007782])

In [50]:
new_predictions.conf_int(alpha = 0.05)

array([[29.02529896, 30.61989124],
       [25.2652456 , 26.47553397],
       [29.87347656, 31.57680741],
       ...,
       [28.43428101, 29.95684404],
       [27.67960321, 29.11543892],
       [26.41425913, 27.72064482]])

Replicate TSS, RSS, estimated sigma, standard error of b1 and b0

In [33]:
results.params

,0
intercept,34.553841
lstat,-0.950049


In [74]:
n = boston.shape[0]

In [35]:
yPred = model.predict(results.params)

In [55]:
RSS_y = sum((boston['medv'] - yPred)**2)
TSS_x = sum((x - boston['lstat'].mean())**2 for x in boston['lstat'])
TSS_y = sum((x - boston['medv'].mean())**2 for x in boston['medv'])

In [56]:
estimatedSigma = np.sqrt(RSS_y / n)

In [57]:
squaredSE_b0 = estimatedSigma**2 * (1/n + boston['lstat'].mean()**2/TSS_x)
squaredSE_b1 = estimatedSigma**2 / TSS_x
print(np.sqrt(squaredSE_b0))
print(np.sqrt(squaredSE_b1))

0.5615143423158607
0.03865679216883057


In [58]:
t = results.params.intercept / np.sqrt(squaredSE_b1) # ??
t

893.8620858262589

In [59]:
Rsquared = 1 - RSS_y / TSS_y
Rsquared

0.5441462975864801

In [76]:
Cor = sum(((boston['lstat'].iloc[i] - boston['lstat'].mean()) * (boston['medv'].iloc[i] - boston['medv'].mean())) for i in range(0, n))
Cor = Cor / np.sqrt(TSS_x * TSS_y)
Cor**2 # == R^2

0.5441462975864791

In [75]:
p = 1 # {lstat}
F = ((TSS_y - RSS_y) / num_predictors) / (RSS_y / (n - p - 1))
F

601.6178711098961

Plot

In [51]:
def abline(ax, b, m, *args, **kwargs):
    "Add a line with slope m and intercept b to ax"
    xlim = ax.get_xlim()
    ylim = [m * xlim[0] + b, m * xlim[1] + b]
    ax.plot(xlim, ylim, *args, **kwargs)